In [ ]:
!pip install caveclient

In [ ]:
import pandas as pd
import numpy as np
import tqdm

import caveclient

import os

REPO_DIR = f"{os.path.abspath('')}/../"

## Instantiate the CAVE client

In [ ]:
my_token = None

In [ ]:
client = caveclient.CAVEclient("minnie65_phase3_v1", auth_token=my_token)

In [ ]:
mat_version = 1293

## Load proofreading and region data

In [ ]:
proof_df = client.materialize.query_table("proofreading_status_and_strategy", materialization_version=mat_version)
proof_df = proof_df[proof_df["strategy_axon"] != 'none'].reset_index(drop=True)

proof_df["strategy_axon"].value_counts()

In [ ]:
region_df = client.materialize.query_table("nucleus_functional_area_assignment", materialization_version=mat_version)

region_df["tag"].value_counts()

In [ ]:
proof_root_ids = np.array(proof_df["pt_root_id"])
v1_root_ids = np.array(region_df[region_df["tag"] == "V1"]["pt_root_id"])

proof_v1_root_ids = proof_root_ids[np.isin(proof_root_ids, v1_root_ids)]

## Query synapses

In [ ]:
pre_syn_dfs = []
for id_block in tqdm.tqdm(np.array_split(proof_v1_root_ids, len(proof_v1_root_ids) // 50)):
    pre_syn_dfs.append(client.materialize.synapse_query(pre_ids=id_block, materialization_version=mat_version, split_positions=True))

post_syn_dfs = []
for id_block in tqdm.tqdm(np.array_split(proof_v1_root_ids, len(proof_v1_root_ids) // 50)):
    post_syn_dfs.append(client.materialize.synapse_query(post_ids=id_block, materialization_version=mat_version, split_positions=True))


syn_df = pd.concat(pre_syn_dfs + post_syn_dfs).drop_duplicates("id", keep="first").reset_index(drop=True)

## Reduce and store

In [ ]:
syn_df = syn_df[["id", "ctr_pt_position_x", "ctr_pt_position_y", "ctr_pt_position_z", "pre_pt_root_id", "post_pt_root_id", "size"]].copy()
syn_df = syn_df.astype({"id": np.int32, "ctr_pt_position_x": np.int32, "ctr_pt_position_y": np.int32, "ctr_pt_position_z": np.int32, "size": np.int32})

syn_df = syn_df.sort_values(["post_pt_root_id", "pre_pt_root_id"]).reset_index(drop=True)

In [ ]:
syn_df.to_feather(f"{REPO_DIR}/data/synapses_proofread_V1_neurons_{mat_version}.feather", compression="zstd", compression_level=9)